### Przygotowanie danych

Naszym pierwszym zadaniem jest przygotowanie wsadu dla modeli uczenia maszynowego. Zrobimy to przy wykorzystaniu `numpy`, `pandas` i `rasterio`, generując szybko ramkę danych z kolumnami `NDVI`, `NDWI`, `SCI` i kolumną `y`, która reprezentować będzie dane binarne: **to, czy dany piksel jest terenem zabudowanym, czy nie jest**.

In [1]:
import numpy as np
import pandas as pd
import rasterio as rio

from sklearn.model_selection import train_test_split

from bands.load import LoadBandsFromTif

In [2]:
BINARY_DATA = './results/clc_results/binarized_clc.tif'
NDVI_DATA = './results/ndvi.tif'
NDWI_DATA = './results/ndwi.tif'
SCI_DATA = './results/sci.tif'

Teraz ułożymy wartości do ramki danych, najpierw wczytamy wszystkie dane:

In [3]:
bin_band = LoadBandsFromTif(BINARY_DATA).get_band()
ndv_band = LoadBandsFromTif(NDVI_DATA).get_band()
ndw_band = LoadBandsFromTif(NDWI_DATA).get_band()
sci_band = LoadBandsFromTif(SCI_DATA).get_band()

Teraz utworzymy ramkę danych... najpierw wszystkie wartości:

In [4]:
arr = np.empty(shape=(len(bin_band.flatten()), 4))
arr.shape

(4445532, 4)

In [5]:
arr[:, 0] = bin_band.flatten()
arr[:, 1] = ndv_band.flatten()
arr[:, 2] = ndw_band.flatten()
arr[:, 3] = sci_band.flatten()

In [6]:
columns = ['y', 'ndvi', 'ndwi', 'sci']
df = pd.DataFrame(data=arr, columns=columns)
df.head()

,y,ndvi,ndwi,sci
0,0.0,0.861824,-0.827907,0.827907
1,0.0,0.857498,-0.818329,0.818329
2,0.0,0.857490,-0.826433,0.826433
3,0.0,0.868816,-0.825994,0.825994
4,0.0,0.865950,-0.835684,0.835684


Wczytamy podstawowe informacje o danych:

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4445532 entries, 0 to 4445531
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   y       float64
 1   ndvi    float64
 2   ndwi    float64
 3   sci     float64
dtypes: float64(4)
memory usage: 135.7 MB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
y,4445532.0,0.586137,0.492525,0.000000,0.000000,1.000000,1.000000,1.000000
ndvi,4445532.0,0.519922,0.254621,-0.740552,0.307663,0.571663,0.751201,1.000000
ndwi,4445532.0,-0.557063,0.207317,-1.000000,-0.727110,-0.611056,-0.411924,0.715823
sci,4445532.0,0.557063,0.207317,-0.715823,0.411924,0.611056,0.727110,1.000000


Dane wyglądają na prawidłowe.
Zapiszemy dataframe do pliku csv:

In [9]:
df.to_csv('./results/prepared_frame.csv')

### Transformacja danych wejściowych i podział na zbiór treningowy i testowy.

W tym kroku przygotowujemy dane jako wsad do modeli uczenia maszynowego. Jako że wskaźniki są już w przedziale -1 do 1, możemy tylko podzielić dane na zbiór treningowy i testowy. Wykorzystamy do tego możliwości, jakie daje pakiet `scikit-learn`.

In [10]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop('y', axis=1),
    df['y'],
    random_state=14)

In [11]:
print(x_train.describe().T)
print("------------------------------------------------------------------------------")
print(x_test.describe().T)

          count      mean       std       min       25%       50%       75%  \
ndvi  3334149.0  0.519918  0.254616 -0.740552  0.307630  0.571672  0.751152   
ndwi  3334149.0 -0.557048  0.207348 -1.000000 -0.727063 -0.611058 -0.411840   
sci   3334149.0  0.557048  0.207348 -0.715823  0.411840  0.611058  0.727063   

           max  
ndvi  1.000000  
ndwi  0.715823  
sci   1.000000  
------------------------------------------------------------------------------
          count      mean       std       min       25%       50%       75%  \
ndvi  1111383.0  0.519933  0.254637 -0.667292  0.307802  0.571625  0.751345   
ndwi  1111383.0 -0.557110  0.207225 -1.000000 -0.727244 -0.611053 -0.412178   
sci   1111383.0  0.557110  0.207225 -0.626313  0.412178  0.611053  0.727244   

           max  
ndvi  1.000000  
ndwi  0.626313  
sci   1.000000  


In [21]:
print("Zbiór y trening liczy", y_train.count(), "elementów.")
print("Zbiór y testowy liczy", y_test.count(), "elementów.")

Zbiór y trening liczy 3334149 elementów.
Zbiór y testowy liczy 1111383 elementów.
